# Chap02 - 텐서플로 설치에서 실행까지

## 2.1 텐서프로 설치

```bash
# TensorFlow CPU 
$ pip install tensorflow

# TensorFlow GPU
$ pip install tensorflow-gpu
```

## 2.2 Hello World

TensorFlow의 첫 번째 예제인 `Hello`와 `World!`라는 단어를 결합해 `Hello World!`라는 문구를 출력하는 프로그램이다. 

In [1]:
import tensorflow as tf

# TensorFlow 버전 확인
print(tf.__version__)

1.8.0


In [2]:
# hello_world.py

h = tf.constant("Hello")  # 상수
w = tf.constant(" World!")
hw = h + w

with tf.Session() as sess:
    ans = sess.run(hw)
    
print(ans)

b'Hello World!'


다음은 `str` 타입의 `phw = ph + pw`와 `tf.constant` 타입의 `hw`를 비교해보자.

In [3]:
ph = "Hello"
pw = " World!"
phw = ph + pw

print('str: {}, type: {}'.format(phw, type(phw)))
print('tensorflow: {}, type: {}'.format(hw, type(hw)))

str: Hello World!, type: <class 'str'>
tensorflow: Tensor("add:0", shape=(), dtype=string), type: <class 'tensorflow.python.framework.ops.Tensor'>


3장에서 텐서플로의 연산 그래프 모델을 자세히 알아 볼 수 있다.

텐서플로의 연산 그래프와 관련된 중요한 개념은, 

1. 먼저 어떠한 연산을 할지 정의해둔 후
2. 외부 메커니즘을 통해서 그 연산을 실행 시킨다.

위의 코드에서 `hw = h + w`에서 텐서플로 코드는 `h`와 `w`의 합인 `Hello World!`를 출력하는 것이 아닌 나중에 실행될 합 연산을 **연산 그래프(computation graph)** 에 추가한다.

그런 다음, `Session` 객체는 텐서플로 연산 메커니즘에 대한 인터페이스 역할을 하며, `ans = sess.run(hw)`에서 `hw = h + w`연산을 수행한다. 따라서, `print(ans)`를 하면 `Hllo World!`를 출력한다.

## 2.3 MNIST

위에서 간단한 텐서플로 예제를 살펴 보았다. 이번에는 필기체 숫자로 이루어진 'MNIST 데이터베이스'를 이용해 필기체 숫자 분류를 해보자. MNIST는 미국 인구조사국으 지기원들이 쓴 숫자와 고등학생들이 쓴 숫자로 만든 미국 국립표준기술연구소(NIST)의 데이터베이스를 다시 섞어 만든 필기체 숫자 이미지 데이터베이스이다. 

MNIST 데이터는 딥러닝 예제에서 빠지지 않고 등장하는 데이터라고 할 수 있다. 

![](./images/mnist.png)

## 2.4 소프트맥스 회귀 - Softmax Regression

이번에 구현할 MNIST 분류 예제에서는 **[소프트맥스 회귀(softamx regression)](https://houxianxu.github.io/2015/04/23/logistic-softmax-regression/)** 을 이용하여 구현한다. 텐서플로를 공부하는 것이기 때문에 자세한 설명은 생략한다. [softmax regression](http://ufldl.stanford.edu/tutorial/supervised/SoftmaxRegression/)은 binary classification에서 사용되는 [logistic regression](https://ratsgo.github.io/machine%20learning/2017/04/02/logistic/)을 multinomial classification로 확장한 개념이라 할 수 있다. 자세한 내용은 해당 용어의 링크를 참고하면 된다.

$$
h(x^{(i)}) = \frac{e^{w_{y_j}^Tx^{(i)}}} {\sum_{j = 1}^k e^{w_j^Tx^{(i)}}}
$$

In [2]:
# softmax.py
# import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


DATA_DIR = '../data'
NUM_STEPS = 1000
MINIBATCH_SIZE = 100

data = input_data.read_data_sets(DATA_DIR, one_hot=True)

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [4]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))

y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x ,W)

# Cost(Loss) Function
cross_entropy = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y_true))

# Gradient Step(Optimze)
gd_step = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

In [7]:
with tf.Session() as sess:
    # 학습
    sess.run(tf.global_variables_initializer())
    
    for _ in range(NUM_STEPS):
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})
        
    ans = sess.run(accuracy, feed_dict={x: data.test.images,
                                       y_true: data.test.labels})
    
print("Accuracy: {:.4f}".format(ans*100))

Accuracy: 91.1200
